<a href="https://colab.research.google.com/github/menotti/pp/blob/master/code/OpenCL/TutorialOpenCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/menotti/playground-eb8w0m4e

Cloning into 'playground-eb8w0m4e'...
remote: Enumerating objects: 477, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 477 (delta 4), reused 0 (delta 0), pack-reused 465
Receiving objects: 100% (477/477), 242.74 KiB | 15.17 MiB/s, done.
Resolving deltas: 100% (243/243), done.


#Configurações necessárias para executar OpenCL no Collab



Como os drivers mais recentes não suportam OpenCL, é preciso instalar um driver da nvidia mais antigo que suporte.
ps: essa operação leva em torno de uns 3 minutos

Você pode verificar que não funciona, executando
`! clinfo`, que deve retornar `Number of platforms                               0` antes da configuração

In [ ]:
!clinfo

Number of platforms                               1
  Platform Name                                   NVIDIA CUDA
  Platform Vendor                                 NVIDIA Corporation
  Platform Version                                OpenCL 3.0 CUDA 12.0.151
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_fp64 cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_icd cl_nv_compiler_options cl_nv_device_attribute_query cl_nv_pragma_unroll cl_nv_copy_opts cl_nv_create_buffer cl_khr_int64_base_atomics cl_khr_int64_extended_atomics cl_nv_kernel_attribute cl_khr_device_uuid cl_khr_pci_bus_info cl_khr_external_semaphore cl_khr_external_memory cl_khr_external_semaphore_opaque_fd cl_khr_external_memory_opaque_fd
  Platform Host timer resolution                  0ns
  Platform Extens

In [ ]:
!sudo apt update
!sudo apt purge *nvidia* -y
!sudo DEBIAN_FRONTEND=noninteractive  apt install nvidia-driver-530 -y

Streaming output truncated to the last 5000 lines.
Package 'linux-objects-nvidia-418-server-5.4.0-91-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-92-generic' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-92-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-94-generic' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-94-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-96-generic' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-96-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-97-generic' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-97-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-418-server-5.4.0-99-generic' is not installed, so not removed
Package 'linux

#Introdução ao OpenCL

Este repositório contem o material usado para o tutorial OpenCL hospedado no site Tech.io:

[Introdução ao OpenCL-tech.io](https://tech.io/playgrounds/54224/introducao-ao-opencl/)

Traduzido e adaptado do original em inglês disponível em:

[Introduction to OpenCL](https://tech.io/playgrounds/48226/introduction-to-sycl/)

##Introdução ao OpenCL


###O que é OpenCL?

O OpenCL (Open Computing Language) é um padrão aberto e isento de royalties para programação paralela em múltiplas plataformas de diversos aceleradores encontrados em supercomputadores, servidores em nuvem, computadores pessoais, dispositivos móveis e plataformas embarcadas. O OpenCL pode melhorar o desempenho de um amplo espectro de aplicações de várias areas, incluindo ferramentas criativas profissionais, software científico e médico, processamento de imagens e treinamento e inferências em redes neurais.

Este tutorial tem como objetivo ensinar os fundamentos do OpenCL por meio da explicação de uma aplicação simples.

Usaremos o uma implementação compatível com OpenCL 1.2 para compilar e executar os exemplos.

1 Este tutorial faz parte de um [Curso de Programação Paralela](https://github.com/menotti/pp)  e traz apenas exemplos de uso. Acesse o material completo para mais detalhes.

##Começando com OpenCL

###Como o OpenCL funciona?

Uma implementação OpenCL consiste em dois componentes principais: (i) um compilador que compila seu código para dispositivos OpenCL, e (ii) uma biblioteca de tempo de execução que fornece a interface de alto nível para escrever aplicativos OpenCL e o runtime para executar o código do dispositivo em dispositivos OpenCL.

###Em que o OpenCL é executado?

O OpenCL pode atingir uma ampla variedade de dispositivos em qualquer sistema, como CPUs com vários núcleos, GPUs, FPGAs, DSPs e outros tipos de aceleradores e processadores especializados.

O pacote OpenCL fornece uma ferramenta chamada `clinfo`, que pode ser usada para detectar dispositivos suportados no sistema.

Para fins de demonstração, podemos executar o `clinfo` aqui para exibir os dispositivos OpenCL disponíveis para este tutorial.

Clique no botão `Run` para ver como é sua saída:

```
! sh /content/playground-eb8w0m4e/cpp-project/validate.sh
```

Observe que este tutorial está sendo executado em uma instância de nuvem e, portanto, está usando uma CPU Intel, não há GPU ou processador acelerador disponível.
Como você pode ver na saída de comando `clinfo`, o único dispositivo suportado é o "host", que é a CPU Intel. Embora seja improvável oferecer melhorias de desempenho na ausência de uma GPU, ele permite executar nosso código OpenCL.

In [ ]:
! sh /content/playground-eb8w0m4e/cpp-project/validate.sh

lscpu output:
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.170
BogoMIPS:                        4000.34
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1


## Olá mundo



Como de costume em qualquer tutorial de programação, vamos começar por este [olá mundo](https://us.fixstars.com/products/opencl/book/OpenCLProgrammingBook/first-opencl-program/) adaptado de um livro gratuito disponível na internet. O programa não faz nenhum cálculo, apenas retorna a mensagem **"Hello, World!"** gerada no _kernel_. Como a entrada e saída padrão do sistema não podem ser usadas a partir do _kernel_, ele apenas vai preencher a cadeia de caracteres e enviá-la ou programa principal que irá imprimi-la na console.

###Incluindo o arquivo de cabeçalho OpenCL

Toda aplicação OpenCL deve incluir o arquivo de cabeçalho `CL/cl.h`:

```c
#include <CL/cl.h>
```

Mas antes disso, nós estamos definindo `CL_TARGET_OPENCL_VERSION 120` para usar a versão 1.2 da API sem gerar _warnings_ durante a compilação:

```c
#define CL_TARGET_OPENCL_VERSION 120
```

Esta versão não é a mais recente, mas é suportada pela maioria dos dispositivos OpenCL.

### Configuração no _host_

No `main`, nós definimos uma série de variáveis cujos tipos, definidos na biblioteca OpenCL, possuem o prefixo `cl_` que explicaremos a seguir:

```c
cl_device_id device_id = NULL;
cl_context context = NULL;
cl_command_queue command_queue = NULL;
cl_mem memobj = NULL;
cl_program program = NULL;
cl_kernel kernel = NULL;
cl_platform_id platform_id = NULL;
cl_uint ret_num_devices;
cl_uint ret_num_platforms;
cl_int ret;
```

### Configurando uma fila (Queue) OpenCL

Para enviar nossas tarefas para serem agendadas e executadas no dispositivo de destino, precisamos usar uma fila (Queue) OpenCL associada a um dispositivo. Primeiro pegamos o primeiro dispositivo da primeira plataforma disponível:

```
/* Get Platform and Device Info */
ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
ret = clGetDeviceIDs(platform_id, CL_DEVICE_TYPE_DEFAULT, 1, &device_id, &ret_num_devices);
```

Depois criamos um contexto de execução e uma fila associada e este contexto:
```
/* Create OpenCL context */
context = clCreateContext(NULL, 1, &device_id, NULL, NULL, &ret);

/* Create Command Queue */
command_queue = clCreateCommandQueue(context, device_id, 0, &ret);
```
É possível criar contextos com vários dispositivos, mas cada fila está associada e um único dispositivo.

### Configurar armazenamento do dispositivo

Na maioria dos sistemas, o _host_ e o dispositivo não compartilham memória física. Por exemplo, a CPU pode usar RAM e a GPU pode usar sua própria VRAM interna. O OpenCL precisa saber quais dados serão compartilhados entre o _host_ e os dispositivos.

Para esse fim, existem _buffers_ OpenCL. A função a seguir cria um _buffer_ que pode ser lido e escrito (`CL_MEM_READ_WRITE`).

```c
/* Create Memory Buffer */
memobj = clCreateBuffer(context, CL_MEM_READ_WRITE,MEM_SIZE * sizeof(char), NULL, &ret);
```

### Compilando o _kernel_

Os próximos passos são criar (`clCreateProgramWithSource`) e compilar (`clBuildProgram`) um programa que resultará em uma bibliteca dinâmica para o dispositivo alvo. Nosso único _kernel_ está armazenado em um arquivo externo (`hello.cl`), mas ele poderia conter mais de um, então precisamos indicar o seu nome `hello` como argumento na função `clCreateKernel`:

```c
/* Create Kernel Program from the source */
program = clCreateProgramWithSource(context, 1, (const char **)&source_str,
(const size_t *)&source_size, &ret);

/* Build Kernel Program */
ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

/* Create OpenCL Kernel */
kernel = clCreateKernel(program, "hello", &ret);
```

### Executando o _kernel_

Agora basta atribuir o argumento que será usado no _kernel_ e enfileirar sua execução na fila do dispositivo:

```c
/* Set OpenCL Kernel Parameters */
ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&memobj);

/* Execute OpenCL Kernel */
ret = clEnqueueTask(command_queue, kernel, 0, NULL,NULL);
```

Nosso _kernel_ não recebe nenhum dado do _host_, mas quando for necessário, é preciso enviar os dados antes da execução usando `clEnqueueWriteBuffer`.

### Lendo e exibindo o resultado

Após a execução, enfileiramos a transferência dos dados do dispositivo para _host_ com a função `clEnqueueReadBuffer` que tem como destino a `string` declarada no _host_ e a exibimos:

```c
/* Copy results from the memory buffer */
ret = clEnqueueReadBuffer(command_queue, memobj, CL_TRUE, 0,
MEM_SIZE * sizeof(char),string, 0, NULL, NULL);

/* Display Result */
puts(string);
```

### Limpando

Como estamos usando a versão C da API, precisamos desalocar os recursos:

```c
/* Finalization */
ret = clFlush(command_queue);
ret = clFinish(command_queue);
ret = clReleaseKernel(kernel);
ret = clReleaseProgram(program);
ret = clReleaseMemObject(memobj);
ret = clReleaseCommandQueue(command_queue);
ret = clReleaseContext(context);

free(source_str);
```

Isso pode não ser necessário em linguagens de programação mais modernas como C++, Python e Java.

### Vamos executar!


In [ ]:
%%writefile /content/playground-eb8w0m4e/cpp-project/src/exercises/hello_world.cpp
#include <stdio.h>
#include <stdlib.h>

#define CL_TARGET_OPENCL_VERSION 120


#include <CL/cl.h>

#define MEM_SIZE (128)
#define MAX_SOURCE_SIZE (0x100000)

int main()
{
    cl_device_id device_id = NULL;
    cl_context context = NULL;
    cl_command_queue command_queue = NULL;
    cl_mem memobj = NULL;
    cl_program program = NULL;
    cl_kernel kernel = NULL;
    cl_platform_id platform_id = NULL;
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret;

    char string[MEM_SIZE];

    FILE *fp;
    char fileName[] = "../content/playground-eb8w0m4e/cpp-project/src/exercises/kernels/hello.cl";
    char *source_str;
    size_t source_size;

    /* Load the source code containing the kernel*/
    fp = fopen(fileName, "r");
        if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread(source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose(fp);

    /* Get Platform and Device Info */
    ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs(platform_id, CL_DEVICE_TYPE_DEFAULT, 1, &device_id, &ret_num_devices);

    /* Create OpenCL context */
    context = clCreateContext(NULL, 1, &device_id, NULL, NULL, &ret);

    /* Create Command Queue */
    command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    /* Create Memory Buffer */
    memobj = clCreateBuffer(context, CL_MEM_READ_WRITE,MEM_SIZE * sizeof(char), NULL, &ret);

    /* Create Kernel Program from the source */
    program = clCreateProgramWithSource(context, 1, (const char **)&source_str,
    (const size_t *)&source_size, &ret);

    /* Build Kernel Program */
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    /* Create OpenCL Kernel */
    kernel = clCreateKernel(program, "hello", &ret);

    /* Set OpenCL Kernel Parameters */
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&memobj);

    /* Execute OpenCL Kernel */
    ret = clEnqueueTask(command_queue, kernel, 0, NULL,NULL);

    /* Copy results from the memory buffer */
    ret = clEnqueueReadBuffer(command_queue, memobj, CL_TRUE, 0,
    MEM_SIZE * sizeof(char),string, 0, NULL, NULL);

    /* Display Result */
    puts(string);

    /* Finalization */
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(memobj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);

    free(source_str);

    return 0;
}


Overwriting /content/playground-eb8w0m4e/cpp-project/src/exercises/hello_world.cpp


In [ ]:
! g++ -Wno-deprecated /content/playground-eb8w0m4e/cpp-project/src/exercises/hello_world.cpp -o /content/playground-eb8w0m4e/hello_world -lOpenCL -I.

In [ ]:
! ../content/playground-eb8w0m4e/hello_world

Hello, World!


## Um exemplo completo



### Multiplicação de matrizes

Agora vamos analisar um exemplo completo de multiplicação de matrizes em C++ que envia dados para processamento e os recebe de volta. Além das diferenças inerentes da linguagem, temos algumas novidades em relação ao nosso exemplo anterior.

O _kernel_ é criado na seguinte linha:

```c++
// Create the compute kernel from the program
cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer> naive_mmul(program, "mmul");
```

Depois, ele é invocado diretamente pelo seu nome, mas antes declaramos um objeto `NDRange` que estipula suas dimensões e tamanhos:

```c++
cl::NDRange global(N, N);
naive_mmul(cl::EnqueueArgs(queue, global), N, d_a, d_b, d_c);
```
Os _kernels_ OpenCL podem trabalhar com 1, 2 ou 3 dimensões. Neste exemplo, estamos usando 2 dimensões e cada instância do _kernel_ vai calcular um único elemento da matriz. Observe no código do _kernel_ que está na última aba as seguintes linhas:

```c
int i = get_global_id(0);
int j = get_global_id(1);
```
A função `get_global_id` retorna um identificador único em cada dimensão (`0` e `1`) para cada uma das N x N execuções, cobrindo toda a matriz.

In [ ]:
%%writefile /content/playground-eb8w0m4e/cpp-project/src/exercises/device_picker.hpp
/*------------------------------------------------------------------------------
 *
 * Name:       device_picker.h
 *
 * Purpose:    Provide a simple CLI to specify an OpenCL device at runtime
 *
 * Note:       Must be included AFTER the relevant OpenCL header
 *             See one of the Matrix Multiply exercises for usage
 *
 * HISTORY:    Method written by James Price, October 2014
 *             Extracted to a common header by Tom Deakin, November 2014
 */

#pragma once

#include <vector>
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/err_code.h"
#include <iostream>

#define MAX_INFO_STRING 256


unsigned getDeviceList(std::vector<cl::Device>& devices)
{
  cl_int err;

  // Get list of platforms
  std::vector<cl::Platform> platforms;
  cl::Platform::get(&platforms);

  // Enumerate devices
  for (int i = 0; i < platforms.size(); i++)
  {
    cl_uint num = 0;
    std::vector<cl::Device> plat_devices;
    platforms[i].getDevices(CL_DEVICE_TYPE_ALL, &plat_devices);
    devices.insert(devices.end(), plat_devices.begin(), plat_devices.end());
  }

  return devices.size();
}

void getDeviceName(cl::Device& device, std::string& name)
{
  cl_device_info info = CL_DEVICE_NAME;

  // Special case for AMD
#ifdef CL_DEVICE_BOARD_NAME_AMD
  device.getInfo(CL_DEVICE_VENDOR, &name);
  if (strstr(name.c_str(), "Advanced Micro Devices"))
    info = CL_DEVICE_BOARD_NAME_AMD;
#endif

  device.getInfo(info, &name);
}


int parseUInt(const char *str, cl_uint *output)
{
  char *next;
  *output = strtoul(str, &next, 10);
  return !strlen(next);
}

void parseArguments(int argc, char *argv[], cl_uint *deviceIndex)
{
  for (int i = 1; i < argc; i++)
  {
    if (!strcmp(argv[i], "--list"))
    {
      // Get list of devices
      std::vector<cl::Device> devices;
      unsigned numDevices = getDeviceList(devices);

      // Print device names
      if (numDevices == 0)
      {
        std::cout << "No devices found.\n";
      }
      else
      {
        std::cout << "\nDevices:\n";
        for (int i = 0; i < numDevices; i++)
        {
          std::string name;
          getDeviceName(devices[i], name);
          std::cout << i << ": " << name << "\n";
        }
        std::cout << "\n";
      }
      exit(0);
    }
    else if (!strcmp(argv[i], "--device"))
    {
      if (++i >= argc || !parseUInt(argv[i], deviceIndex))
      {
        std::cout << "Invalid device index\n";
        exit(1);
      }
    }
    else if (!strcmp(argv[i], "--help") || !strcmp(argv[i], "-h"))
    {
      std::cout << "\n";
      std::cout << "Usage: ./program [OPTIONS]\n\n";
      std::cout << "Options:\n";
      std::cout << "  -h  --help               Print the message\n";
      std::cout << "      --list               List available devices\n";
      std::cout << "      --device     INDEX   Select device at INDEX\n";
      std::cout << "\n";
      exit(0);
    }
  }
}



Overwriting /content/playground-eb8w0m4e/cpp-project/src/exercises/device_picker.hpp


In [ ]:
%%writefile /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2d.cpp
//------------------------------------------------------------------------------
//
//  PROGRAM: Matrix Multipliplication driver
//
//  PURPOSE: This is a driver program to test various ways of computing
//           the product:
//
//                C  = A * B
//
//           A and B are set to constant matrices so we
//           can make a quick test of the multiplication.
//
//  USAGE:   The matrices are constant matrices, square and the order is
//           set as a constant, ORDER (see mult.h).
//
//  HISTORY: Written by Tim Mattson, August 2010
//           Modified by Simon McIntosh-Smith, September 2011
//           Modified by Tom Deakin and Simon McIntosh-Smith, October 2012
//           Updated to C++ Wrapper v1.2.6 by Tom Deakin, August 2013
//           Modified to assume square matricies by Tom Deakin, October 2014
//
//------------------------------------------------------------------------------


#define CL_TARGET_OPENCL_VERSION 120

#include <iostream>
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matmul.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.cpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/util.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/err_code.h"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/device_picker.hpp"

int main(int argc, char *argv[])
{

    int N;   // A[N][N], B[N][N], C[N][N]
    int size;      // Number of elements in each matrix

    double start_time;      // Starting time
    double run_time;        // Timing data
    util::Timer timer;      // timing

    N = ORDER;

    size = N * N;

    std::vector<float> h_A(size); // Host memory for Matrix A
    std::vector<float> h_B(size); // Host memory for Matrix B
    std::vector<float> h_C(size); // Host memory for Matrix C

    cl::Buffer d_a, d_b, d_c;   // Matrices in device memory

//--------------------------------------------------------------------------------
// Create a context and queue
//--------------------------------------------------------------------------------

    try
    {
        cl_uint deviceIndex = 0;
        parseArguments(argc, argv, &deviceIndex);

        // Get list of devices
        std::vector<cl::Device> devices;
        unsigned numDevices = getDeviceList(devices);

        // Check device index in range
        if (deviceIndex >= numDevices)
        {
          std::cout << "Invalid device index (try '--list')\n";
          return EXIT_FAILURE;
        }

        cl::Device device = devices[deviceIndex];

        std::string name;
        getDeviceName(device, name);
        std::cout << "\nUsing OpenCL device: " << name << "\n";

        std::vector<cl::Device> chosen_device;
        chosen_device.push_back(device);
        cl::Context context(chosen_device);
        cl::CommandQueue queue(context, device);

//--------------------------------------------------------------------------------
// Setup the buffers, initialize matrices, and write them into global memory
//--------------------------------------------------------------------------------

        //  Reset A, B and C matrices (just to play it safe)
        initmat(N, h_A, h_B, h_C);

        d_a = cl::Buffer(context, h_A.begin(), h_A.end(), true);
        d_b = cl::Buffer(context, h_B.begin(), h_B.end(), true);
        d_c = cl::Buffer(context, CL_MEM_WRITE_ONLY, sizeof(float) * size);

//--------------------------------------------------------------------------------
// OpenCL matrix multiplication ... Naive
//--------------------------------------------------------------------------------

        // Create the compute program from the source buffer
        cl::Program program(context, util::loadProgram("../content/playground-eb8w0m4e/cpp-project/src/exercises/kernels/C_elem.cl"), true);

        // Create the compute kernel from the program
        cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer> naive_mmul(program, "mmul");

        printf("\n===== OpenCL, matrix mult, C(i,j) per work item, order %d ======\n",N);

        // Do the multiplication COUNT times
        for (int i = 0; i < COUNT; i++)
        {
            zero_mat(N, h_C);

            start_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0;

            // Execute the kernel over the entire range of C matrix elements ... computing
            // a dot product for each element of the product matrix.  The local work
            // group size is set to NULL ... so I'm telling the OpenCL runtime to
            // figure out a local work group size for me.
            cl::NDRange global(N, N);
            naive_mmul(cl::EnqueueArgs(queue, global),
                    N, d_a, d_b, d_c);

            queue.finish();

            run_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0 - start_time;

            cl::copy(queue, d_c, h_C.begin(), h_C.end());

            results(N, h_C, run_time);

        } // end for loop

    } catch (cl::Error err)
    {
        std::cout << "Exception\n";
        std::cerr << "ERROR: "
                  << err.what()
                  << "("
                  << err_code(err.err())
                  << ")"
                  << std::endl;
    }

    return EXIT_SUCCESS;
}


Overwriting /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2d.cpp


In [ ]:
! g++ -Wno-deprecated /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2d.cpp -o /content/playground-eb8w0m4e/matmul2d -lOpenCL -I.

In [ ]:
! ../content/playground-eb8w0m4e/matmul2d


Using OpenCL device: Tesla T4

===== OpenCL, matrix mult, C(i,j) per work item, order 1024 ======
 0.10 seconds at 22605.1 MFLOPS 
 0.09 seconds at 22605.1 MFLOPS 
 0.04 seconds at 52377.6 MFLOPS 
 0.04 seconds at 53687.1 MFLOPS 
 0.04 seconds at 52377.6 MFLOPS 
 0.04 seconds at 53687.1 MFLOPS 
 0.04 seconds at 59652.3 MFLOPS 
 0.04 seconds at 59652.3 MFLOPS 
 0.04 seconds at 61356.7 MFLOPS 
 0.04 seconds at 61356.7 MFLOPS 


## Usando memória local

### Multiplicação de matrizes

Aqui temos implementações diferentes, onde cada invocação do _kernel_ calcula uma linha inteira da matriz. Note que nosso `NDRange` agora só tem uma dimensão e que no kernel (`C_row.cl`) só solicitamos o `get_global_id` para ela, já que a outra é percorrida por um `for` na variável `k`. Esta primeira versão apresenta desempenho pior do que a da lição anterior, ao menos na CPU. Experimente as outras duas versões desta lição alternando os comentários no programa principal. Note que na última versão (`C_row_priv_block.cl`) você precisa declarar um `NDRange` local e passa-lo como argumento no _kernel_, pois ele usa as funções `get_local_id` e `get_local_size`, além de alocar memória local.

In [ ]:
%%writefile /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul1d.cpp

//------------------------------------------------------------------------------
//
//  PROGRAM: Matrix Multipliplication driver
//
//  PURPOSE: This is a driver program to test various ways of computing
//           the product:
//
//                C  = A * B
//
//           A and B are set to constant matrices so we
//           can make a quick test of the multiplication.
//
//  USAGE:   The matrices are constant matrices, square and the order is
//           set as a constant, ORDER (see mult.h).
//
//  HISTORY: Written by Tim Mattson, August 2010
//           Modified by Simon McIntosh-Smith, September 2011
//           Modified by Tom Deakin and Simon McIntosh-Smith, October 2012
//           Updated to C++ Wrapper v1.2.6 by Tom Deakin, August 2013
//           Modified to assume square matricies by Tom Deakin, October 2014
//
//------------------------------------------------------------------------------

#define CL_TARGET_OPENCL_VERSION 120

#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matmul.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.cpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/util.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/err_code.h"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/device_picker.hpp"

int main(int argc, char *argv[])
{

    int N;   // A[N][N], B[N][N], C[N][N]
    int size;      // Number of elements in each matrix

    double start_time;      // Starting time
    double run_time;        // Timing data
    util::Timer timer;      // timing

    N = ORDER;

    size = N * N;

    std::vector<float> h_A(size); // Host memory for Matrix A
    std::vector<float> h_B(size); // Host memory for Matrix B
    std::vector<float> h_C(size); // Host memory for Matrix C

    cl::Buffer d_a, d_b, d_c;   // Matrices in device memory

//--------------------------------------------------------------------------------
// Create a context and queue
//--------------------------------------------------------------------------------

    try
    {
        cl_uint deviceIndex = 0;
        parseArguments(argc, argv, &deviceIndex);

        // Get list of devices
        std::vector<cl::Device> devices;
        unsigned numDevices = getDeviceList(devices);

        // Check device index in range
        if (deviceIndex >= numDevices)
        {
          std::cout << "Invalid device index (try '--list')\n";
          return EXIT_FAILURE;
        }

        cl::Device device = devices[deviceIndex];

        std::string name;
        getDeviceName(device, name);
        std::cout << "\nUsing OpenCL device: " << name << "\n";

        std::vector<cl::Device> chosen_device;
        chosen_device.push_back(device);
        cl::Context context(chosen_device);
        cl::CommandQueue queue(context, device);

//--------------------------------------------------------------------------------
// Setup the buffers, initialize matrices, and write them into global memory
//--------------------------------------------------------------------------------

        //  Reset A, B and C matrices (just to play it safe)
        initmat(N, h_A, h_B, h_C);

        d_a = cl::Buffer(context, h_A.begin(), h_A.end(), true);
        d_b = cl::Buffer(context, h_B.begin(), h_B.end(), true);
        d_c = cl::Buffer(context, CL_MEM_WRITE_ONLY, sizeof(float) * size);

//--------------------------------------------------------------------------------
// OpenCL matrix multiplication ... C row per work item
// OpenCL matrix multiplication ... C row per work item, A row in pivate memory
// OpenCL matrix multiplication ... C row per work item, A row pivate, B col local
//--------------------------------------------------------------------------------

        // Create the compute program from the source buffer
        cl::Program program(context, util::loadProgram("../content/playground-eb8w0m4e/cpp-project/src/exercises/kernels/C_row.cl"), true);
//         cl::Program program(context, util::loadProgram("kernels/C_row_priv.cl"), true);
//         cl::Program program(context, util::loadProgram("kernels/C_row_priv_bloc.cl"), true);

        // Create the compute kernel from the program
        cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer> crow_mmul(program, "mmul");
//         cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer> arowpriv_mmul(program, "mmul");
//         cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer, cl::LocalSpaceArg> browloc_mmul(program, "mmul");

        printf("\n===== OpenCL, matrix mult, C row per work item, order %d ======\n",N);
//         printf("\n===== OpenCL, matrix mult, C row, A row in priv mem, order %d ======\n",N);
//         printf("\n===== OpenCL, mat mult, C row, priv A, B cols loc, order %d ======\n",N);

        // Do the multiplication COUNT times
        for (int i = 0; i < COUNT; i++)
        {
            zero_mat(N, h_C);

            start_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0;

            cl::NDRange global(N);
//            cl::NDRange local(ORDER / 16);
//            cl::LocalSpaceArg localmem = cl::Local(sizeof(float) * N);

            crow_mmul(cl::EnqueueArgs(queue, global), N, d_a, d_b, d_c);
//            arowpriv_mmul(cl::EnqueueArgs(queue, global, local), N, d_a, d_b, d_c);
//            browloc_mmul(cl::EnqueueArgs(queue, global, local), N, d_a, d_b, d_c, localmem);

            queue.finish();

            run_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0 - start_time;

            cl::copy(queue, d_c, h_C.begin(), h_C.end());

            results(N, h_C, run_time);

        } // end for loop

    } catch (cl::Error err)
    {
        std::cout << "Exception\n";
        std::cerr << "ERROR: "
                  << err.what()
                  << "("
                  << err_code(err.err())
                  << ")"
                  << std::endl;
    }

    return EXIT_SUCCESS;
}


Overwriting /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul1d.cpp


In [ ]:
! g++ -Wno-deprecated /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul1d.cpp -o /content/playground-eb8w0m4e/matmul1d -lOpenCL -I.

In [ ]:
! ../content/playground-eb8w0m4e/matmul1d


Using OpenCL device: Tesla T4

===== OpenCL, matrix mult, C row per work item, order 1024 ======
 0.32 seconds at 6690.0 MFLOPS 
 0.19 seconds at 11126.9 MFLOPS 
 0.19 seconds at 11608.0 MFLOPS 
 0.18 seconds at 11608.0 MFLOPS 
 0.18 seconds at 11608.0 MFLOPS 
 0.19 seconds at 11608.0 MFLOPS 
 0.19 seconds at 11608.0 MFLOPS 
 0.19 seconds at 11608.0 MFLOPS 
 0.19 seconds at 11608.0 MFLOPS 
 0.18 seconds at 11671.1 MFLOPS 


## Blocagem de _loops_

### Multiplicação de matrizes

Por fim, temos a versão com blocagem, que explora melhor o uso das memórias cache. Experimente alterar o tamanho do bloco e compare os resultados, lembre-se que ele deve ser alterado no programa principal e também no _kernel_.

In [ ]:
%%writefile /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2db.cpp

//------------------------------------------------------------------------------
//
//  PROGRAM: Matrix Multipliplication driver
//
//  PURPOSE: This is a driver program to test various ways of computing
//           the product:
//
//                C  = A * B
//
//           A and B are set to constant matrices so we
//           can make a quick test of the multiplication.
//
//  USAGE:   The matrices are constant matrices, square and the order is
//           set as a constant, ORDER (see mult.h).
//
//  HISTORY: Written by Tim Mattson, August 2010
//           Modified by Simon McIntosh-Smith, September 2011
//           Modified by Tom Deakin and Simon McIntosh-Smith, October 2012
//           Updated to C++ Wrapper v1.2.6 by Tom Deakin, August 2013
//           Modified to assume square matricies by Tom Deakin, October 2014
//
//------------------------------------------------------------------------------

#define CL_TARGET_OPENCL_VERSION 120

#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matmul.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/matrix_lib.cpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/util.hpp"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/err_code.h"
#include "../content/playground-eb8w0m4e/cpp-project/src/exercises/device_picker.hpp"

int main(int argc, char *argv[])
{

    int N;   // A[N][N], B[N][N], C[N][N]
    int size;      // Number of elements in each matrix

    double start_time;      // Starting time
    double run_time;        // Timing data
    util::Timer timer;      // timing

    N = ORDER;

    size = N * N;

    std::vector<float> h_A(size); // Host memory for Matrix A
    std::vector<float> h_B(size); // Host memory for Matrix B
    std::vector<float> h_C(size); // Host memory for Matrix C

    cl::Buffer d_a, d_b, d_c;   // Matrices in device memory

//--------------------------------------------------------------------------------
// Create a context and queue
//--------------------------------------------------------------------------------

    try
    {
        cl_uint deviceIndex = 0;
        parseArguments(argc, argv, &deviceIndex);

        // Get list of devices
        std::vector<cl::Device> devices;
        unsigned numDevices = getDeviceList(devices);

        // Check device index in range
        if (deviceIndex >= numDevices)
        {
          std::cout << "Invalid device index (try '--list')\n";
          return EXIT_FAILURE;
        }

        cl::Device device = devices[deviceIndex];

        std::string name;
        getDeviceName(device, name);
        std::cout << "\nUsing OpenCL device: " << name << "\n";

        std::vector<cl::Device> chosen_device;
        chosen_device.push_back(device);
        cl::Context context(chosen_device);
        cl::CommandQueue queue(context, device);

//--------------------------------------------------------------------------------
// Setup the buffers, initialize matrices, and write them into global memory
//--------------------------------------------------------------------------------

        //  Reset A, B and C matrices (just to play it safe)
        initmat(N, h_A, h_B, h_C);

        d_a = cl::Buffer(context, h_A.begin(), h_A.end(), true);
        d_b = cl::Buffer(context, h_B.begin(), h_B.end(), true);
        d_c = cl::Buffer(context, CL_MEM_WRITE_ONLY, sizeof(float) * size);

//--------------------------------------------------------------------------------
// OpenCL matrix multiplication ... blocked
//--------------------------------------------------------------------------------

        // Create the compute program from the source buffer
        cl::Program program(context, util::loadProgram("../content/playground-eb8w0m4e/cpp-project/src/exercises/kernels/C_block_form.cl"), true);

        // Create the compute kernel from the program
        cl::make_kernel<int, cl::Buffer, cl::Buffer, cl::Buffer, cl::LocalSpaceArg, cl::LocalSpaceArg> block_mmul(program, "mmul");

        // Work-group computes a block of C.  This size is also set
        // in a #define inside the kernel function.  Note this blocksize
        // must evenly divide the matrix order
        int blocksize = 16;

        printf("\n===== Parallel matrix mult (blocked, size=%d), order %d on device ======\n", blocksize, ORDER);

        // Do the multiplication COUNT times
        for (int i = 0; i < COUNT; i++)
        {
            zero_mat(N, h_C);

            start_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0;

            cl::LocalSpaceArg A_block = cl::Local(sizeof(float) * blocksize*blocksize);
            cl::LocalSpaceArg B_block = cl::Local(sizeof(float) * blocksize*blocksize);

            block_mmul(
                cl::EnqueueArgs(
                    queue,
                    cl::NDRange(N,N),
                    cl::NDRange(blocksize,blocksize)),
                N,
                d_a,
                d_b,
                d_c,
                A_block,
                B_block);

            queue.finish();

            run_time = static_cast<double>(timer.getTimeMilliseconds()) / 1000.0 - start_time;

            cl::copy(queue, d_c, h_C.begin(), h_C.end());

            results(N, h_C, run_time);

        } // end for loop

    } catch (cl::Error err)
    {
        std::cout << "Exception\n";
        std::cerr << "ERROR: "
                  << err.what()
                  << "("
                  << err_code(err.err())
                  << ")"
                  << std::endl;
    }

    return EXIT_SUCCESS;
}


Overwriting /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2db.cpp


In [ ]:
! g++ -Wno-deprecated /content/playground-eb8w0m4e/cpp-project/src/exercises/matmul2db.cpp -o /content/playground-eb8w0m4e/matmul2db -lOpenCL -I.

In [ ]:
! ../content/playground-eb8w0m4e/matmul2db


Using OpenCL device: Tesla T4

===== Parallel matrix mult (blocked, size=16), order 1024 on device ======
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 268435.5 MFLOPS 
 0.01 seconds at 268435.5 MFLOPS 
 0.01 seconds at 306783.4 MFLOPS 
 0.01 seconds at 268435.5 MFLOPS 


## Onde aprender mais sobre o OpenCL?

### Documentação Khronos Group

A melhor fonte de informações sobre OpenCL é o [site do consórcio](https://www.khronos.org/opencl/), onde podem ser encontrados guias de referência, exemplos de código, livros, vídeos, apresentações e outros materiais.

### Tutorial Hands On OpenCL

Boa parte do código usado neste tutorial foi baseado neste [outro tutorial](http://handsonopencl.github.io/).

### Livro da Fixstars Corporation

Também foram usados partes de código [deste livro](https://us.fixstars.com/products/opencl/book/OpenCLProgrammingBook/contents/) sobre o assunto, disponível gratuitamente.